In [7]:
from pathlib import Path
import pandas as pd


In [8]:
# Categorías a analizar
categories = pd.read_csv("../../data/categories/categories_stoten.csv", sep=";", header=None)
categories_list = categories.iloc[:, 0].astype(str).str.upper().tolist()

# Cogemos inferencia de llava
llava_classification = pd.read_csv("../../data/inference/stoten_w_descriptions.csv", sep=";", header=0)
llava_classification["img"] = llava_classification["img"].apply(lambda x: x.split("/")[-1])
llava_classification = llava_classification[["img","category_llava"]]
llava_classification['category_llava'] = llava_classification['category_llava'].apply(lambda x: x.upper())
# Las categorías que no están en la lista de categorías, han sido malas inferencias. 

# 1. Hay varias categorías que se pueden afinar. Por ejemplo, el modelo ha puesto SPIRITUAL, y no la frase completa
# de Spiritual, symbolic and related connotations
llava_classification["category_llava"] = llava_classification["category_llava"].replace("SPIRITUAL","SPIRITUAL, SYMBOLIC AND RELATED CONNOTATIONS") 
llava_classification["category_llava"] = llava_classification["category_llava"].replace("SYMBOLIC AND RELATED CONNOTATIONS","SPIRITUAL, SYMBOLIC AND RELATED CONNOTATIONS") 

llava_classification["category_llava"] = llava_classification["category_llava"].apply(
                    lambda cat: cat if cat in categories_list or cat == "NOT VALID" or cat == "NOT RELEVANT" else "BAD_INFERENCE"
                    )

# Mergeamos con el resultado de etiquetado manual
manual_classification = pd.read_csv("../../data/inference/stoten_manual_annotation.csv", sep=";", header=0)
# ATENCIÓN, HAY NAs NO CLASIFICADO EN STOTEN. Ponemos Other type
manual_classification['manual_category'] = manual_classification['manual_category'].fillna("Other type")
manual_classification['manual_category'] = manual_classification['manual_category'].apply(lambda x: x.upper())
# Unimos ambos por imagen
result = llava_classification.merge(manual_classification[["img","manual_category"]],on="img",how="left")



print(result.head(10).to_string())
result.to_csv("lvlm_vs_manual.csv",sep=";")



                                       img      category_llava     manual_category
0  aiguestortes_NA_1366_47701956991__b.jpg        RECREATIONAL  NATURE & LANDSCAPE
1  aiguestortes_NA_1633_31399087577__b.jpg  NATURE & LANDSCAPE       FAUNA & FLORA
2  aiguestortes_NA_1633_50177463812__b.jpg        RECREATIONAL              SPORTS
3  aiguestortes_NA_1690_52491627395__b.jpg  NATURE & LANDSCAPE  NATURE & LANDSCAPE
4  aiguestortes_NA_2008_19242320949__b.jpg  NATURE & LANDSCAPE       FAUNA & FLORA
5  aiguestortes_NA_4531_49900773177__b.jpg        BIRDWATCHING       FAUNA & FLORA
6  aiguestortes_NA_4792_24046167316__b.jpg       FAUNA & FLORA        NOT RELEVANT
7    guadarrama_NA_1089_20943542884__b.jpg               URBAN       RURAL TOURISM
8    guadarrama_NA_1089_21378498778__b.jpg            CULTURAL           RELIGIOUS
9    guadarrama_NA_1273_52582432385__b.jpg            CULTURAL        NOT RELEVANT


In [9]:
nan_rows = result[result.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [img, category_llava, manual_category]
Index: []


In [11]:
from sklearn.metrics import classification_report

# Generar el reporte de clasificación
report = classification_report(
    result['manual_category'], 
    result['category_llava'], 
    output_dict=True, 
    zero_division=0
)

# Convertir a DataFrame y transponer para que las categorías sean el índice
df_report = pd.DataFrame(report).T

# # Mostrar el DataFrame con categorías a la izquierda y métricas como columnas
# df_report = df_report.drop(columns=["support"], errors="ignore")
df_report = df_report.drop(index=["accuracy","macro avg","weighted avg","nan"], errors="ignore")
# df_report = df_report.dropna(axis=0, how="all")
print(df_report.to_string())
df_report = df_report.round(3)
df_report.to_csv("lvlm_vs_manual_report.csv", sep=";")


                    precision    recall  f1-score  support
BIRDWATCHING         0.000000  0.000000  0.000000      0.0
CULTURAL             0.241441  0.491743  0.323867    545.0
FAUNA & FLORA        0.973948  0.360267  0.525974   1349.0
GASTRONOMY           0.876543  0.489655  0.628319    290.0
NATURE & LANDSCAPE   0.520161  0.739962  0.610892   1569.0
NOT RELEVANT         0.000000  0.000000  0.000000   1301.0
OTHER TYPE           0.000000  0.000000  0.000000     42.0
RECREATIONAL         0.027778  0.308989  0.050973    178.0
RELIGIOUS            0.853107  0.396325  0.541219    381.0
RURAL TOURISM        0.537604  0.309295  0.392675    624.0
SPORTS               0.750000  0.009494  0.018750    632.0
SUN & BEACH          0.172414  0.089286  0.117647     56.0
URBAN                0.123007  0.514286  0.198529    105.0
